In [1]:
from imblearn.over_sampling import SMOTE
import os
import cv2
import numpy as np

# Define the directories for benign and malignant images
benign_dir = '/home/vuk/Documents/ML_Data/HAM/train/benign'
malignant_dir = '/home/vuk/Documents/ML_Data/HAM/train/malignant'

# Initialize lists to store images and labels
images = []
labels = []


In [2]:
nan_indices = np.isnan(images).any(axis=1)
print("Number of samples containing NaN values:", np.sum(nan_indices))

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:

# Function to load and preprocess images
def load_and_preprocess_images(directory):
    for filename in os.listdir(directory):
        img = cv2.imread(os.path.join(directory, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to grayscale
        img = img.flatten() #
        images.append(img)
        labels.append(0 if 'benign' in directory else 1) # Assign labels
    return np.array(images), np.array(labels)

# Load and preprocess images
X_benign, y_benign = load_and_preprocess_images(benign_dir)
X_malignant, y_malignant = load_and_preprocess_images(malignant_dir)

# Combine benign and malignant images and labels
X = np.concatenate((X_benign, X_malignant))
y = np.concatenate((y_benign, y_malignant))

# Apply SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert oversampled data back to images
# Assuming the images were originally grayscale and resized to 64x64
X_resampled_images = X_resampled.reshape(-1, 600, 450)

# Save oversampled images back to the malignant directory
for i, img in enumerate(X_resampled_images):
    # Ensure the directory exists
    if not os.path.exists(malignant_dir):
        os.makedirs(malignant_dir)
    # Save the image
    cv2.imwrite(os.path.join(malignant_dir, f'malignant_{i}.png'), img)
